In [1]:
"""
eval_no_viewer_gtx1080ti.py

Optimized for GTX 1080 Ti (11GB VRAM, CUDA 6.1):
- Reduced memory footprint for CLIP
- FP32 precision (GTX 1080 Ti doesn't efficiently support FP16/BF16)
- Batch processing with gradient disabled
- CPU fallback options
"""

import numpy as np
from scipy.spatial import cKDTree
import trimesh
from PIL import Image, ImageDraw
import torch
import clip
import sys
import gc

sys.argv = [x for x in sys.argv if x != "-f"]


# -------------------------
# Geometry: Chamfer Distance
# -------------------------
def chamfer_distance(mesh1, mesh2, n_samples=5000):
    """Compute Chamfer Distance with chunked processing for large point clouds"""
    try:
        pcd1 = mesh1.sample(n_samples)
        pcd2 = mesh2.sample(n_samples)
    except Exception as e:
        print(f"Warning: Sampling failed ({e}), using vertices directly")
        pcd1 = mesh1.vertices[:n_samples]
        pcd2 = mesh2.vertices[:n_samples]

    # Use chunked processing for very large point clouds
    chunk_size = 10000
    if len(pcd1) > chunk_size or len(pcd2) > chunk_size:
        print(f"Using chunked Chamfer Distance computation...")
        return chamfer_distance_chunked(pcd1, pcd2, chunk_size)
    
    tree1 = cKDTree(pcd1)
    tree2 = cKDTree(pcd2)

    d12, _ = tree1.query(pcd2, k=1)
    d21, _ = tree2.query(pcd1, k=1)

    cd = float(np.mean(d12) + np.mean(d21))
    return cd


def chamfer_distance_chunked(pcd1, pcd2, chunk_size=10000):
    """Chunked version for memory efficiency"""
    tree1 = cKDTree(pcd1)
    tree2 = cKDTree(pcd2)
    
    # Query in chunks
    d12_list = []
    for i in range(0, len(pcd2), chunk_size):
        chunk = pcd2[i:i+chunk_size]
        d, _ = tree1.query(chunk, k=1)
        d12_list.append(d)
    d12 = np.concatenate(d12_list)
    
    d21_list = []
    for i in range(0, len(pcd1), chunk_size):
        chunk = pcd1[i:i+chunk_size]
        d, _ = tree2.query(chunk, k=1)
        d21_list.append(d)
    d21 = np.concatenate(d21_list)
    
    cd = float(np.mean(d12) + np.mean(d21))
    return cd


# -------------------------
# Topology: non-manifold ratio
# -------------------------
def non_manifold_ratio(mesh):
    """Robust non-manifold edge detection"""
    try:
        edges = mesh.edges_unique
        edge_faces = mesh.edges_face
        
        if edge_faces is None or len(edges) == 0:
            return 0.0
        
        edge_faces = np.atleast_2d(edge_faces)
        
        # Count non-manifold edges (missing faces or multiple faces)
        if edge_faces.ndim == 1:
            edge_faces = edge_faces.reshape(-1, 1)
        
        non_manifold = np.sum(np.any(edge_faces == -1, axis=1))
        ratio = float(non_manifold) / max(1, len(edges))
        return ratio
    except Exception as e:
        print(f"Warning: Non-manifold computation failed ({e}), returning 0.0")
        return 0.0


# -------------------------
# CPU renderer for CLIP
# -------------------------
def render_pointcloud_views(mesh, image_size=224, point_radius=2, views=3):
    """
    CPU-based orthographic projection rendering
    Optimized for memory efficiency
    """
    try:
        verts = mesh.vertices.copy()
    except Exception as e:
        print(f"Warning: Could not access vertices ({e})")
        return [Image.new("RGB", (image_size, image_size), (255, 255, 255))] * views
    
    if len(verts) == 0:
        return [Image.new("RGB", (image_size, image_size), (255, 255, 255))] * views
    
    # Center and normalize
    center = verts.mean(axis=0)
    verts -= center
    
    scale = np.max(np.linalg.norm(verts, axis=1))
    if scale <= 0:
        scale = 1.0
    verts = verts / scale
    
    imgs = []
    projections = {
        "front": verts[:, [0, 1]],
        "side": verts[:, [2, 1]],
        "top": verts[:, [0, 2]],
    }
    
    selected = ["front", "side", "top"][:views]
    for k in selected:
        pts = projections[k]
        margin = int(0.05 * image_size)
        coords = ((pts + 1.0) * 0.5 * (image_size - 2*margin)) + margin
        coords = np.round(coords).astype(int)
        coords[:, 0] = np.clip(coords[:, 0], 0, image_size-1)
        coords[:, 1] = np.clip(coords[:, 1], 0, image_size-1)
        
        img = Image.new("RGB", (image_size, image_size), (255, 255, 255))
        draw = ImageDraw.Draw(img)
        
        # Draw points with anti-aliasing effect
        for (x, y) in coords:
            x0 = x - point_radius
            y0 = image_size - 1 - y - point_radius
            x1 = x + point_radius
            y1 = image_size - 1 - y + point_radius
            draw.ellipse([x0, y0, x1, y1], fill=(30, 30, 30))
        
        imgs.append(img)
    
    return imgs


# -------------------------
# CLIP similarity - GTX 1080 Ti optimized
# -------------------------
def clip_similarity_from_meshes(mesh1, mesh2, image_size=224, device=None, use_cpu=False):
    """
    CLIP similarity optimized for GTX 1080 Ti
    - Reduced image size option
    - CPU fallback
    - Explicit memory management
    """
    if device is None:
        if use_cpu or not torch.cuda.is_available():
            device = "cpu"
            print("Using CPU for CLIP (slower but safer for memory)")
        else:
            device = "cuda"
            print(f"Using GPU: {torch.cuda.get_device_name(0)}")
    
    try:
        # Clear GPU cache before loading
        if device == "cuda":
            torch.cuda.empty_cache()
            gc.collect()
        
        # Load CLIP model with FP32 (GTX 1080 Ti works best with FP32)
        print("Loading CLIP model...")
        model, preprocess = clip.load("ViT-B/32", device=device, jit=False)
        model.eval()  # Ensure eval mode
        
        # Render views
        print("Rendering views...")
        imgs1 = render_pointcloud_views(mesh1, image_size=image_size)
        imgs2 = render_pointcloud_views(mesh2, image_size=image_size)
        
        def combine(imgs):
            widths, heights = zip(*(i.size for i in imgs))
            total_w = sum(widths)
            max_h = max(heights)
            out = Image.new("RGB", (total_w, max_h), (255, 255, 255))
            x_offset = 0
            for im in imgs:
                out.paste(im, (x_offset, 0))
                x_offset += im.size[0]
            return out
        
        comb1 = combine(imgs1)
        comb2 = combine(imgs2)
        
        # Preprocess images
        image1 = preprocess(comb1).unsqueeze(0).to(device)
        image2 = preprocess(comb2).unsqueeze(0).to(device)
        
        # Compute similarity with no gradient
        with torch.no_grad():
            e1 = model.encode_image(image1).float()  # Explicit FP32
            e2 = model.encode_image(image2).float()
            
            # Normalize
            e1 = e1 / e1.norm(dim=-1, keepdim=True)
            e2 = e2 / e2.norm(dim=-1, keepdim=True)
            
            # Compute similarity
            sim = float((e1 @ e2.T).cpu().item())
        
        # Cleanup
        del model, image1, image2, e1, e2
        if device == "cuda":
            torch.cuda.empty_cache()
        gc.collect()
        
        return sim
        
    except RuntimeError as e:
        if "out of memory" in str(e):
            print("\n⚠️ GPU Out of Memory! Retrying with CPU...")
            if device == "cuda":
                torch.cuda.empty_cache()
                gc.collect()
            return clip_similarity_from_meshes(mesh1, mesh2, image_size, "cpu", True)
        else:
            raise e


# -------------------------
# Main evaluation
# -------------------------
def evaluate_models(obj1_path, obj2_path, n_samples=3000, use_cpu_clip=False):
    """
    Evaluate two OBJ models
    
    Args:
        obj1_path: Path to first OBJ file
        obj2_path: Path to second OBJ file
        n_samples: Number of points for Chamfer Distance (reduce if OOM)
        use_cpu_clip: Force CPU for CLIP computation
    """
    print(f"\n{'='*60}")
    print("3D Model Evaluation - GTX 1080 Ti Optimized")
    print(f"{'='*60}\n")
    
    # Check CUDA availability
    if torch.cuda.is_available():
        print(f"✓ CUDA available: {torch.cuda.get_device_name(0)}")
        print(f"✓ CUDA version: {torch.version.cuda}")
        print(f"✓ Available memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB\n")
    else:
        print("⚠️ CUDA not available, using CPU\n")
    
    # Load meshes
    print(f"Loading models...")
    print(f"  Model 1: {obj1_path}")
    print(f"  Model 2: {obj2_path}")
    
    try:
        mesh1 = trimesh.load(obj1_path, force='mesh')
        mesh2 = trimesh.load(obj2_path, force='mesh')
        print(f"✓ Models loaded successfully")
        print(f"  Model 1: {len(mesh1.vertices)} vertices, {len(mesh1.faces)} faces")
        print(f"  Model 2: {len(mesh2.vertices)} vertices, {len(mesh2.faces)} faces\n")
    except Exception as e:
        print(f"❌ Error loading models: {e}")
        return None
    
    results = {}
    
    # 1. Chamfer Distance
    print(f"\n{'▶ '*30}")
    print("1. Geometry: Chamfer Distance")
    print(f"{'▶ '*30}")
    try:
        cd = chamfer_distance(mesh1, mesh2, n_samples=n_samples)
        print(f"✓ Chamfer Distance = {cd:.6f}")
        results['chamfer'] = cd
    except Exception as e:
        print(f"❌ Chamfer Distance failed: {e}")
        results['chamfer'] = None
    
    # 2. Non-manifold ratio
    print(f"\n{'▶ '*30}")
    print("2. Topology: Non-manifold Edge Ratio")
    print(f"{'▶ '*30}")
    try:
        nm1 = non_manifold_ratio(mesh1)
        nm2 = non_manifold_ratio(mesh2)
        print(f"✓ Model 1 non-manifold ratio = {nm1:.4f}")
        print(f"✓ Model 2 non-manifold ratio = {nm2:.4f}")
        results['nm1'] = nm1
        results['nm2'] = nm2
    except Exception as e:
        print(f"❌ Non-manifold computation failed: {e}")
        results['nm1'] = None
        results['nm2'] = None
    
    # 3. CLIP similarity
    print(f"\n{'▶ '*30}")
    print("3. Perceptual: CLIP Similarity")
    print(f"{'▶ '*30}")
    try:
        clip_score = clip_similarity_from_meshes(
            mesh1, mesh2, 
            image_size=224,
            use_cpu=use_cpu_clip
        )
        print(f"✓ CLIP Similarity = {clip_score:.4f}")
        results['clip'] = clip_score
    except Exception as e:
        print(f"❌ CLIP computation failed: {e}")
        results['clip'] = None
    
    # Summary
    print(f"\n{'='*60}")
    print("Summary")
    print(f"{'='*60}")
    for k, v in results.items():
        if v is not None:
            print(f"  {k:15s}: {v:.6f}")
        else:
            print(f"  {k:15s}: FAILED")
    print(f"{'='*60}\n")
    
    return results


# -------------------------
# Entry point
# -------------------------
if __name__ == "__main__":
    import sys
    
    # Parse arguments
    if len(sys.argv) >= 3:
        obj1 = sys.argv[1]
        obj2 = sys.argv[2]
    else:
        obj1 = "3d_eval_project/models/tripo_ai/hair dry.obj"
        obj2 = "3d_eval_project/models/hunyuan3D/hair hunyuan.obj"
    
    # Optional: force CPU for CLIP if you encounter memory issues
    use_cpu = "--cpu" in sys.argv
    
    # Optional: adjust sample count
    n_samples = 3000
    for arg in sys.argv:
        if arg.startswith("--samples="):
            n_samples = int(arg.split("=")[1])
    
    if use_cpu:
        print("⚠️ Forcing CPU mode for CLIP\n")
    
    # Run evaluation
    evaluate_models(obj1, obj2, n_samples=n_samples, use_cpu_clip=use_cpu)

/home/hpc/iwfa/iwfa120h/miniforge3/envs/eval3d-clean/lib/python3.11/site-packages/clip/clip.py:6: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import packaging



3D Model Evaluation - GTX 1080 Ti Optimized

✓ CUDA available: NVIDIA GeForce GTX 1080 Ti
✓ CUDA version: 11.8
✓ Available memory: 11.71 GB

Loading models...
  Model 1: 3d_eval_project/models/tripo_ai/hair dry.obj
  Model 2: 3d_eval_project/models/hunyuan3D/hair hunyuan.obj
✓ Models loaded successfully
  Model 1: 414278 vertices, 783386 faces
  Model 2: 840587 vertices, 1499542 faces


▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ 
1. Geometry: Chamfer Distance
▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ 
✓ Chamfer Distance = 0.510184

▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ 
2. Topology: Non-manifold Edge Ratio
▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ 
✓ Model 1 non-manifold ratio = 0.0000
✓ Model 2 non-manifold ratio = 0.0000

▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ 
3. Perceptual: CLIP Similarity
▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ ▶ 
Using GPU: NVIDIA GeForce GTX 1080 Ti